Importing the neccessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report,roc_curve,roc_auc_score
from sklearn.ensemble import AdaBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC

!pip install xgboost
from xgboost import XGBClassifier

plt.rcParams['figure.figsize']=(15,10)
plt.rcParams['figure.dpi']=300
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load The DataSet
df=pd.read_csv("/content/sensor.csv")
df

In [ ]:
df.describe()

In [ ]:
df.describe(include=object)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.drop("Unnamed: 0",inplace=True,axis=1)


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

The column "sensor_15" has only null values so we dropped tye column


In [ ]:
df.drop("sensor_15",inplace=True, axis=1)

In [ ]:
sns.heatmap(df.isnull(),cmap="viridis")
plt.tight_layout()

in the dataset most of the columns contains missing values

to replace the missing values check for outliers

Since the columns are numerical

if outliers are present replace the missing values with the median
otherwise replace the missing values with the mean

In [ ]:
sns.boxplot(df,palette="rainbow")
plt.tight_layout()

In [ ]:
df.hist()
plt.tight_layout()

In [ ]:
for i in df.isna().sum():
  print(i/len(df)*100)

In [ ]:
# prompt: replace missing values with the median for all numerical columns

for col in df.select_dtypes(include=np.number).columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)


In [ ]:
 df.isnull().sum()

There are 3 categorical columns in the Data set, Convert them into numerical

In [ ]:
df["machine_status"].value_counts()

In [ ]:
le=LabelEncoder()
df["machine_status"]=le.fit_transform(df["machine_status"])

In [ ]:
df["machine_status"].value_counts()

In [ ]:
sns.boxplot(df,palette="rainbow")
plt.tight_layout()

# to treat the outliers
capping replacing outlier values is called capping
In capping all outlier values will be replaced by upper extreame or lower extreame
Outlier detection : user defined function to calculate upper extreame and lower extreame

In [ ]:


def calculate_extremes(df):
    for col in df.select_dtypes(include=np.number).columns:
        q1 = df[col].quantile(0.25)
        q2 = df[col].quantile(0.5)  # Quartile 2 (Median)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        upper_extreame = q3 + (1.5 * iqr)
        lower_extreame = q1 - (1.5 * iqr)

        print(f"Column: {col}")
        print(f"Quartile 1 (Q1): {q1}")
        print(f"Quartile 2 (Q2/Median): {q2}")
        print(f"Interquartile Range (IQR): {iqr}")
        print(f"Upper Extreme: {upper_extreame}")
        print(f"Lower Extreme: {lower_extreame}")
        print("-" * 20)

calculate_extremes(df)


In [ ]:

def treat_outliers(df):
    for col in df.select_dtypes(include=np.number).columns:
        if col != 'machine_status':  # Exclude 'machine_status' column
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_extreme = q3 + (1.5 * iqr)
            lower_extreme = q1 - (1.5 * iqr)

            df[col] = np.where(df[col] > upper_extreme, upper_extreme, df[col])
            df[col] = np.where(df[col] < lower_extreme, lower_extreme, df[col])
    return df

df = treat_outliers(df)


In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df["machine_status"].value_counts()

In [ ]:
sns.countplot(x='machine_status', data=df,palette="viridis")
plt.title('Machine Status')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(df,palette="rainbow")
plt.tight_layout()

In [ ]:
df.hist(color="yellow",edgecolor="black")
plt.tight_layout()

In [ ]:


# Create the plot.
for column in df.columns:
    if column != 'machine_status':
        plt.figure(figsize=(10, 6))
        plt.plot(df.index, df[column], label=column)
        plt.xlabel('Time')
        plt.ylabel('Sensor Reading')
        plt.title(f'Sensor {column} Readings Over Time')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(),annot=True,cmap="RdYlGn")
plt.tight_layout()

In [ ]:
x = df.drop(["machine_status", "timestamp"], axis=1)
y = df["machine_status"]

In [ ]:
x

In [ ]:
y

In [ ]:

sc=StandardScaler()
x1=sc.fit_transform(x)

In [ ]:
x=pd.DataFrame(x1,columns=x.columns)
x

In [ ]:
x.describe()

# **Model Building**

In [ ]:
# splitting the dataset into training(77%) and testing(33%)data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

## **1.Logistic Regression**

In [ ]:
model1=LogisticRegression()
model1.fit(x_train,y_train)

In [ ]:
y_predict1=model1.predict(x_test)
y_predict1

In [ ]:
df1=pd.DataFrame({"Actual":y_test,"predicted":y_predict1})
df1

In [ ]:
conf_matrix=confusion_matrix(y_test,y_predict1)
print(conf_matrix)

In [ ]:
print("training accuracy:", model1.score(x_train,y_train))
print("testing accuracy:", model1.score(x_test,y_test))

In [ ]:
class_report1=classification_report(y_test,y_predict1)
print(class_report1)

## **2. Decision Tree Classifier**

In [ ]:
model2=DecisionTreeClassifier()
model2.fit(x_train,y_train)

In [ ]:
x.columns

In [ ]:
y.unique()

In [ ]:
fn=x.columns
cn=['2','1','0']
tree.plot_tree(model2,feature_names=fn,class_names=cn,filled=True)
plt.tight_layout()

In [ ]:
y_predict2=model2.predict(x_test)
y_predict2

In [ ]:

y_test.values

In [ ]:
print("train_accuracy:",model2.score(x_train,y_train))
print("test_accuracy:",model2.score(x_test,y_test))

In [ ]:
model2=DecisionTreeClassifier(criterion="gini",max_depth=3,min_samples_split=8,random_state=42)
model2.fit(x_train,y_train)

In [ ]:
fn=x.columns
cn=['2','1','0']
tree.plot_tree(model2,feature_names=fn,class_names=cn,filled=True)
plt.tight_layout()

In [ ]:
y_predict2=model2.predict(x_test)
y_predict2

In [ ]:
y_test.values

In [ ]:
print("train_accuracy:",model2.score(x_train,y_train))
print("test_accuracy:",model2.score(x_test,y_test))

In [ ]:
class_report2=classification_report(y_test,y_predict2)
print(class_report2)

## **3. Random forest Classifier**

In [ ]:
model3=RandomForestClassifier(random_state=2)
model3.fit(x_train,y_train)

In [ ]:
y_predict3=model3.predict(x_test)
y_predict3

In [ ]:
y_test.values

In [ ]:
print('train accuracy:',model3.score(x_train,y_train))
print('test accuracy:',model3.score(x_test,y_test))

4. Gradient boosting Classifier

In [ ]:
model4=GradientBoostingClassifier(learning_rate=0.6,max_depth=3,n_estimators=110)
model4.fit(x_train,y_train)

In [ ]:
y_predict4=model4.predict(x_test)
y_predict4

In [ ]:
y_test.values

In [ ]:
print('train accuracy:',model4.score(x_train,y_train))
print('test accuracy:',model4.score(x_test,y_test))

In [ ]:
class_report4=classification_report(y_test,y_predict4)
print(class_report4)

In [ ]:
df.columns